# Element de base pour HFE 

In [1]:
p = 2
m = 1
q = p^m
K.<alpha> = GF(q)
N = 12
L_N.<a> = GF(q^N)
P = L_N._modulus

In [2]:
x = ['x'+str(k) for k in range(N)]
M = PolynomialRing(L_N,N,x)
I = M.ideal([M(x[i])^q - M(x[i]) for i in range(N)])
Q = M.quotient(I)

R.<X> = PolynomialRing(M)
S = R.quotient(R(P))

variable = sum([M(x[i])*X^i for i in range(N)])

def HFE_representation_polynome(f):
    """
    Entrée: f, un polynôme de L_N[X] qui vérifie les conditions d'HFE
    Sortie: (p0,p1,..,pn), polynôme de Fq[x0,..,xn] tel que 
            f(x0,..,xn) = (p0(x0,..,xn),.., pn(x0,..xn))
    """
    f_prime = R(f) # on regarde f dans l'anneau L_N[x1,...,xN][X]

    f_prime = S(f_prime(variable))
    g =M(f_prime.lift()(a))
    
    f = 0
    for i in g.monomials():
        f+= (X^L_N(g.monomial_coefficient(i)).log(a)*i)
    # on evalue f en x0 + x1*X + .. + xN*X^N 
    # que l'on quotiente par le polynome irr de l'extension 
    # cela nous donne un polynôme de degrée N en X ce qui va nous donner les pi
    p_list = [Q(pi).lift() for pi in list(S(f).lift())]
    
    return p_list

# Mise en place du cryptosystème HFE

In [3]:
def HFE_function(d, number_coeff):
    """
    Entrée: d, degrée de f qui est somme de deux puissances de q
            number_coeff, nombre de coefficient non nul de f au plus
    Sortie: f, polynôme valide pour HFE
    """
    
    coeff = [L_N.random_element() for i in range(number_coeff)]
    f = 0
    lim = abs(floor(log(d,q)-1))
    for i in range(len(coeff)-2):
        theta = randint(0,lim)
        phi = randint(0,lim)
        f += coeff[i] * X^(q^theta + q^phi)
        
    mu_0 = L_N.random_element()
    f += mu_0 + coeff[-1] * X^d 
    return f

In [4]:
def HFE_generates_keys(d, number_coeff):
    f = HFE_function(d, number_coeff)
    
    T.<Y> = PolynomialRing(L_N)

    s = R(T.random_element(1))
    t = R(T.random_element(1))
    
    p_list = HFE_representation_polynome(s(f(t)))
    private_key = (L_N,f,s,t)
    public_key = (K, p_list)
    return public_key, private_key 

In [5]:
# example 1 page 5 Patarin avec le clair
def HFE_redundancy(m):
    """ TO DO """
# example 2 Patarin avec le chiffré
def HFE_redundancy_2(y):
    """ TO DO """
    

In [6]:
def HFE_encryption(m, public_key):
    """
    Entrée: m, on supposera qu'il s'agit d'un élément de (F_q)^n
    Sortie: c, c = HFE(m)
    """
    #redundancy = HFE_redundancy(m)
    K, p_list = public_key
    m = tuple(m)
    return [p(m) for p in p_list]

In [7]:
def HFE_decryption(y, private_key):
    L_N, f, s, t = private_key
    a = L_N([0,1])
    y = sum([y[i]*a**i for i in range(N)])

    list_s = list(s)
    s_inv = list_s[1]**(-1) * (X - list_s[0])
    list_t = list(t)
    t_inv = list_t[1]**(-1) * (X - list_t[0])
        
    ft = s_inv(y)
    
    v = tuple((L_N^N).zero())
    
    roots = (f-ft).roots(multiplicities = False)
    m = [L_N.vector_space(map = false)((t_inv(r))(v)) for r in roots]
   
    return m

In [8]:
t = cputime()
public_key, private_key = HFE_generates_keys(12, 4)
cputime(t)

0.605991

In [60]:
private_key

(Finite Field in a of size 2^12,
 ((a^10 + a^8 + a^7 + a^2 + 1))*X^12 + ((a^10 + a^9 + a^7 + a^4 + a^3))*X^8 + ((a^11 + a^10 + a^9 + a^8 + a^7 + a^5 + a^2 + a + 1))*X^2 + (a^8 + a^6 + a^5 + a^4),
 ((a^11 + a^7 + a^4 + a^3 + a))*X + (a^10 + a^9 + a^7 + a^3 + a),
 ((a^11 + a^9 + a^6))*X + (a^11 + a^9 + a^8 + a^6 + a^5 + a^4 + a^2))

In [74]:
v = L_N.vector_space(map=False)(L_N.random_element())
p = HFE_encryption(v,public_key)
m = HFE_decryption(p, private_key)
m

[(1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0),
 (0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0),
 (1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0)]

In [75]:
v

(1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0)

In [76]:
def solve_sys(S,B):
    def solve_sys_rec(S,expens,i):
        expension = expens.copy()
        if i == 0:
            return expension
        Scopy = S.copy()
        f = Scopy[i-1]
        if f == 0:
            return solve_sys_rec(Scopy,expension,i-1)
        f = f.univariate_polynomial()
        root = f.roots(multiplicities = False)
        key = f.variable_name()
        if (len(root) == 1):
            r = root[0]
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            return solve_sys_rec(new_S,expension,i-1)
        res = []
        for r in root:
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            res += [solve_sys_rec(new_S,expension,i-1)]        
        return res
    
    expension = {B(xi):B(xi) for xi in x}
    l = solve_sys_rec(S, expension, len(S))
    return (l)

In [77]:
def HFE_cryptanalysisI(public_key,p):
    K,Sys = public_key
    N = len(Sys)
    x = Sys[0].variables()
    B = PolynomialRing(K,N,x, order = 'lex')
    Sys = [Sys[i] - p[i] for i in range(N)] + [B(xi)^2-B(xi) for xi in x]
    IS = Ideal(B,Sys)
    Sols = IS.groebner_basis(deg_bound = 3)
    return solve_sys(Sols,B)

In [78]:
HFE_cryptanalysisI(public_key,p)

[[{x0: 0,
   x1: 0,
   x2: 1,
   x3: 0,
   x4: 0,
   x5: 1,
   x6: 1,
   x7: 1,
   x8: 1,
   x9: 0,
   x10: 0,
   x11: 0},
  {x0: 1,
   x1: 0,
   x2: 0,
   x3: 0,
   x4: 1,
   x5: 1,
   x6: 1,
   x7: 1,
   x8: 1,
   x9: 0,
   x10: 0,
   x11: 0}],
 [{x0: 0,
   x1: 1,
   x2: 1,
   x3: 0,
   x4: 0,
   x5: 0,
   x6: 0,
   x7: 0,
   x8: 1,
   x9: 1,
   x10: 0,
   x11: 0},
  {x0: 1,
   x1: 1,
   x2: 0,
   x3: 0,
   x4: 1,
   x5: 0,
   x6: 0,
   x7: 0,
   x8: 1,
   x9: 1,
   x10: 0,
   x11: 0}]]

In [79]:
m

[(1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0),
 (0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0),
 (1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0)]

In [80]:
v

(1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0)